In [1]:
### generator pipeline for large chunks of data --> mem efficient

def pipeline(data):
    yield from (x*x for x in data if x%2 == 0)

nums = range(100)
print("sum of squares of even nums: ", sum(pipeline(nums)))

sum of squares of even nums:  161700


In [3]:
def generate(n):
    for i in range(n):
        yield i

nums = generate(10)

print(list(nums))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


In [4]:
### function call tracer

def trace(fn):
    def wrapper(*args, **kwargs):
        print(f"calling: {fn.__name__} | args: {args}")
        return fn(*args, **kwargs)
    return wrapper

@trace
def add(n1, n2): return n1 + n2

print("res: ", add(5,5))

calling: add | args: (5, 5)
res:  10


In [8]:
### context manager timer

from contextlib import contextmanager
import time

@contextmanager
def timer(i):
    start = time.time()
    yield
    print(f"{i} took {time.time() - start:.5f} seconds")

with timer("adder"):
    print(f"results: {sum(range(1000000000))}")

results: 499999999500000000
adder took 10.32419 seconds


In [9]:
### dynamic attr access

class Config:
    def __init__(self, data) -> None:
        self._data = data
    def __getattr__(self, name) -> None:
        return self._data.get(name, None)
    
cfg = Config({
    "debug"   : True,
    "version" : 1.0
})

print(f"Debug Mode: {cfg.debug}")
print(f"Version: {cfg.version}")

Debug Mode: True
Version: 1.0


In [11]:
### fast fibonacci

from functools import lru_cache

@lru_cache(maxsize=128)
def fib(n):
    return n if n<2 else fib(n-1) + fib(n-2)

print([fib(i) for i in range(100)])

[0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181, 6765, 10946, 17711, 28657, 46368, 75025, 121393, 196418, 317811, 514229, 832040, 1346269, 2178309, 3524578, 5702887, 9227465, 14930352, 24157817, 39088169, 63245986, 102334155, 165580141, 267914296, 433494437, 701408733, 1134903170, 1836311903, 2971215073, 4807526976, 7778742049, 12586269025, 20365011074, 32951280099, 53316291173, 86267571272, 139583862445, 225851433717, 365435296162, 591286729879, 956722026041, 1548008755920, 2504730781961, 4052739537881, 6557470319842, 10610209857723, 17167680177565, 27777890035288, 44945570212853, 72723460248141, 117669030460994, 190392490709135, 308061521170129, 498454011879264, 806515533049393, 1304969544928657, 2111485077978050, 3416454622906707, 5527939700884757, 8944394323791464, 14472334024676221, 23416728348467685, 37889062373143906, 61305790721611591, 99194853094755497, 160500643816367088, 259695496911122585, 420196140727489673, 679891637638612258, 110008777

In [12]:
### dispatch table

opers = {
    "+" : lambda a,b: a+b,
    "*" : lambda a,b: a*b
}

def calc(a, oper, b):
    return opers[oper](a, b)

print(calc(3, "*", 4))
print(calc(10, "+", 10))

12
20


In [13]:
### metaprogramming

class Registry:
    items = {}
    def __init_subclass__(cls) -> None:
        Registry.items[cls.__name__] = cls

class A(Registry) : pass
class B(Registry) : pass

print(f"Registered Classes: {Registry.items}")

Registered Classes: {'A': <class '__main__.A'>, 'B': <class '__main__.B'>}


In [1]:
### basic http server

from http.server import SimpleHTTPRequestHandler, HTTPServer

server = HTTPServer(("localhost", 8000), SimpleHTTPRequestHandler)

try:
    print("serving on: http://localhost:8000")
    server.serve_forever()
except KeyboardInterrupt:
    print("server stopped")
    server.server_close()

serving on: http://localhost:8000


127.0.0.1 - - [24/Dec/2025 22:16:23] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [24/Dec/2025 22:16:30] "GET /memdump.py HTTP/1.1" 200 -
127.0.0.1 - - [24/Dec/2025 22:16:39] "GET /local.py HTTP/1.1" 200 -


server stopped


In [2]:
### simulated text sentiment

def sentiment(text):
    positive = sum(word in text.lower() for word in ["good", "happy", "great", "nice"])
    negative = sum(word in text.lower() for word in ["bad", "sad", "terrible", "awful"])

    return "Positive" if positive > negative else "Negative" if negative > positive else "Neutral"

print(sentiment("I am happy and sad about the nice event."))
print(sentiment("I am sad after the terrible news."))

Positive
Negative


In [4]:
### streaming moving average

def moving(stream, n=3):
    window = []
    for x in stream:
        window.append(x)
        if len(window) > n:
            window.pop(0)
            yield sum(window) / len(window)
    
stream = range(1,21)
print(list(moving(stream=stream)))

[3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0]


In [ ]:
### class creation on the fly

attributes = {
    "x" : 10,
    "y" : 20
}

Point = type("Point", (object,), attributes)

p = Point()
print(p.x, p.y) # type: ignore

10 20


In [20]:
### stream processing

def grep(pattern):
    while True:
        line = (yield)
        if pattern in line:
            print(f"PATTERN MATCHED | {line}")
        else:
            print(f"UNMATCHED LABEL | {line}")

g = grep("ERROR")
next(g)

g.send("ALERT: nominal status")
g.send("ERROR: controls anomaly detected")
g.send("ERROR: sensor anomaly detected")
g.send("PRIORITY: CPU anomaly detected")
g.send("PRIORITY: memory anomaly detected")

UNMATCHED LABEL | ALERT: nominal status
PATTERN MATCHED | ERROR: controls anomaly detected
PATTERN MATCHED | ERROR: sensor anomaly detected
UNMATCHED LABEL | PRIORITY: CPU anomaly detected
UNMATCHED LABEL | PRIORITY: memory anomaly detected
